In [27]:
select_example = input('Path or Example:')
FILE_PATH = None
SAMPLES = {
    1: 'KOAX_20240521_1844',
    2: 'KOAX_20240521_1849',
    3: 'KOAX_20240523_2138',

}
if select_example.isdigit():
    FILE_PATH = f"{'./Samples/'}{SAMPLES[int(select_example)]}" # I don't care if it is out of range, I just want to test the code
else:
    FILE_PATH = select_example
import os
fsize = os.stat(FILE_PATH).st_size
f = open(FILE_PATH, 'rb')
import bz2
import namedstruct as ns
import logging

In [28]:
f.seek(0)

log = logging.getLogger(__name__)

VHR = ns.nstruct(
    (ns.char[6],'Tape'),            # Magic number: AR2V00
    (ns.char[2],'Version'),         # Version number
    (ns.char[1],'End of Tape'),     # End of Tape marker
    (ns.char[3],'Sequence Number'), # Extension number
    (ns.uint32,'Date_d'),           # Date of Volume, days since 1/1/1970
    (ns.uint32,'Time_ms'),          # Time of Volume, milliseconds since midnight
    (ns.char[4],'Station'),         # Station ID (ICAO)
    name='VHR',padding=1,endian='>')
LDM_CR = ns.nstruct(
    (ns.int32,'ctrl_word'),      # Control word - lenth of the compressed data
    (ns.raw,'data'),                       # This data is bz2 compressed when saved
    name='LDM_CR',padding=1,endian='>') # TODO: Need to tell it to compress when packing, save the size to the control word


L2A = ns.nstruct(
    (VHR,'VHR'),                       # Volume Header Record
    (LDM_CR[0],'records'),             # LDM_CR records 
    name='L2A',padding=1,endian='>')

# test = VHR.parse(f.read(24))
fi, __ = L2A.parse(f.read())
# go back to the first LDM_CR
f.seek(24)
while(f.tell() < fsize): # read each LDM_CR into a record and add it to the archive - we do this since the datatype is raw here.
    l = f.tell()
    s = int.from_bytes(f.read(4))
    if(s==b'\xff\xff'):
        break
    f.seek(l)
    d = f.read(s+4)
    if (d == b''):
        log.debug('bad data')
    ldm = LDM_CR.create(d)
    fi.records.append(ldm)


In [29]:

if True:
    f.close() # close the file
    log.setLevel(0)

msg_header_channel_enum = ns.enum('msg_header_channel_enum', None, ns.uint8, bitwise=True,
                    # Note: if both redundent channels are 0s, single channel mode. 
                    SINGLE = 0,  # single channel mode
                    RC_1 = 0x1,    # redundant channel 1
                    RC_2 = 0x2,    # redundant channel 2
                    unused = 0x4,  # unused. 
                    ORDA = 0x8     # ORDA channel (vs legacy)
                    )
# Message header structure 
msg_header = ns.nstruct(
        (ns.uint32[3],'prepad'),                # padding
        (ns.uint16,'message_size'),             # size of message in halfwords
        (msg_header_channel_enum,'channel'),    # actually bit field
        (ns.uint8,'message_type'),              # type of message
        (ns.uint16,'sequence'),                 # sequence number of message
        (ns.uint16,'date'),                     # days since 1/1/1970
        (ns.uint32,'milliseconds'),             # milliseconds since midnight
        # Remember, is these 2 are not 1, then the message is extended and needs to be combined with the next message
        (ns.uint16,'segments'),                 # number of segments in message  
        (ns.uint16,'segment'),                  # segment number of this message
        name='msg_header', padding=1, prepack=ns.packvalue(2432,'message_size'), endian='>') # TODO: Use size of message to determine size of message 

# Base message structure
msg_base = ns.nstruct(
    (msg_header,'header'),
    (ns.raw,'data'),
    name='msg_base',padding=1,endian='>')

# UNCOMPRESSED RECORD data
LDM_UR = ns.nstruct(
    (msg_base[0],'msg'),                # Messages - not sure that this will work since it will hold extended data
    name='LDM_UR',padding=1,endian='>') # TODO: Need to tell it to compress when packing, save the size to the control word

RECORD_HOLDER = ns.nstruct(
    (LDM_UR[0],'ldm'),
    name='RECORD_HOLDER',padding=1,endian='>')




In [30]:
# BASE TYPES
CODE_1 = ns.enum('CODE_1',None, ns.uint8, bitwise=False)
CODE_2 = ns.enum('CODE_1',None, ns.uint16, bitwise=False)
SI_1 = ns.nstruct((ns.uint8,'si'),name='SI_1',padding=1,endian='>')
SI_2 = ns.nstruct((ns.uint16,'si'),name='SI_2',padding=1,endian='>')
SI_4 = ns.nstruct((ns.uint32,'si'),name='SI_4',padding=1,endian='>')
SSI_2 = ns.nstruct((ns.int16,'ssi'),name='sSI_2',padding=1,endian='>')
SSI_4 = ns.nstruct((ns.int16,'ssi'),name='sSI_4',padding=1,endian='>')

In [31]:
# MSG 31 TYPES

msg_31_table = ns.nstruct(
    (ns.raw,'data'), # data
    name='msg_31_table',padding=1,endian='>')

db_hdr = ns.nstruct(      # 33 - MSG 31 - Data Block Header
    (ns.char,'db_type'),
    (ns.char[3],'dm_type'),
    (ns.uint16,'db_size'), # size of the data block (bytes) / reserved
    name='db_hdr',padding=1,endian='>')

t_17a = ns.nstruct(      # 17A - MSG 31 - Data Header Block
    (ns.char[4],'ICAO'), # ICAO of the station
    (ns.uint32,'time_ms'), # time of the observation
    (ns.uint16,'date_d'), # date of the observation
    (ns.uint16,'az_n'), # azimuth # of the observation
    (ns.single,'az_a'), # azimuth angle of the observation (32 bit)
    (CODE_1,'compression'), # compression type, values can be 0-3
    (ns.uint8,'spare_17'), # spare
    (ns.uint16,'rad_len'), # length of the radial data (uncompressed) ubckydibg the header, 9360-14296
    (CODE_1,'az_res_spacing'), # azimuth resolution spacing, 1: 0.5, 2: 1.0
    (CODE_1, 'rad_stat'), # radial status 0-132
    (ns.uint8,'el_n'), # elevation number
    (ns.single,'el_a'), # elevation angle
    (CODE_1, 'rad_spot_blacking'), # radial spot blanking: 0=none, 1=radical, 2=elevation, 4=volume
    (SI_1,'az_index_mode'), #0: none, 1-100-> 0.001 to 1.00 deg
    (ns.uint16,'data_block_count'), # number of data blocks
    (ns.uint32,'dbp_vol_dc'), # data block pointer volume data constant     17E
    (ns.uint32,'dbp_el_dc'), # data block pointer elevation data constant   17F
    (ns.uint32,'dbp_rad_dc'), # data block pointer radial data constant     17H
    (ns.uint32,'dbp_ref_m'), # data block pointer for REF moments       17B, 17I
    (ns.uint32,'dbp_vel_m'), # data block pointer for VEL moments       17B, 17I
    (ns.uint32,'dbp_sw_m'), # data block pointer for SW moments         17B, 17I
    (ns.uint32,'dbp_zdr_m'), # data block pointer for ZDR moments       17B, 17I
    (ns.uint32,'dbp_phi_m'), # data block pointer for PHI moments       17B, 17I
    (ns.uint32,'dbp_rho_m'), # data block pointer for RHO moments       17B, 17I
    (ns.uint32,'dbp_cfp_m'), # data block pointer for CFP moments       17B, 17I
    name='t_17a',padding=1,endian='>')

t_17e = ns.nstruct(      # 17E - MSG 31 - Volume Data Constants
    (db_hdr,'header'),
    (ns.uint8,'v_major'), 
    (ns.uint8,'v_minor'), 
    (ns.single,'lat'), # latitude of the radar
    (ns.single,'lon'), # longitude of the radar
    (SSI_2,'height'), # height of the radar
    (ns.uint16,'fh_h'), # height of the feedhorn
    (ns.single,'r_cal_c'), # calibration constant
    (ns.single,'shv_h_tx'), # horizontal tx power
    (ns.single,'shv_v_tx'), # vertical tx power
    (ns.single,'sdr_cal'), # calibration of zdr
    (ns.single,'dp_i'), # inital dp 
    (ns.uint16,'vcp'), # volume coverage pattern
    (CODE_2,'proc_stat_28'), # processing status options - says uint16, but it is bitwise so
    (ns.uint16,'zdr_bewm'), # zdr bias estimate weighted mean
    (ns.uint8[6],'spare_padding'), # padding - 46-51
    name='t_17e',padding=1,endian='>',base=msg_31_table)

t_17f = ns.nstruct(      # 17F - MSG 31- Elevation Data Constants
    (db_hdr,'header'),
    (SSI_2,'atmos'), # atmospheric attenuation
    (ns.single,'cal_z'), # calibration constant - scaling for signal processor
    name='t_17f',padding=1,endian='>',base=msg_31_table)

t_17h = ns.nstruct(      # 17H - MSG 31 - Radical Data Constants
    (db_hdr,'header'),
    (SI_2,'unabm_r_int'), # unambiguous range interval size
    (ns.single,'noise_h'), # noise level for horizontal
    (ns.single,'noise_v'), # noise level for vertical
    (SI_2, 'nyq'), # nyquist velocity
    (ns.uint16,'rad_flags'), # radial flags
    (ns.single,'cal_const_h'), # calibration constant for horizontal
    (ns.single,'cal_const_v'), # calibration constant for vertical
    name='t_17h',padding=1,endian='>',base=msg_31_table)

t_17i = ns.nstruct(      # 17I - TODO
    (ns.raw,'data'), # data
    name='t_17i',padding=1,endian='>', base=msg_31_table)

t_17i_ref = ns.nstruct(      # 17I - MSG 31 - REF Moments
    (ns.raw,'data'), # data
    base=t_17i,name='t_17i_ref',padding=1,endian='>')

t_17i_vel = ns.nstruct(      # 17I - MSG 31 - VEL Moments
    (ns.raw,'data'), # data
    base=t_17i,name='t_17i_vel',padding=1,endian='>')

t_17i_sw = ns.nstruct(      # 17I - MSG 31 - SW Moments
    (ns.raw,'data'), # data
    base=t_17i,name='t_17i_sw',padding=1,endian='>')

t_17i_zdr = ns.nstruct(      # 17I - MSG 31 - ZDR Moments
    (ns.raw,'data'), # data
    base=t_17i,name='t_17i_zdr',padding=1,endian='>')

t_17i_phi = ns.nstruct(      # 17I - MSG 31 - DP Moments
    (ns.raw,'data'), # data
    base=t_17i,name='t_17i_phi',padding=1,endian='>')

t_17i_rho = ns.nstruct(      # 17I - MSG 31 - CC Moments
    (ns.raw,'data'), # datarho
    base=t_17i,name='t_17i_rho',padding=1,endian='>')

t_17i_cfp = ns.nstruct(      # 17I - MSG 31 - Clutter Filter Pwoeer Removed - Moments
    (ns.raw,'data'), # data
    base=t_17i,name='t_17i_cfp',padding=1,endian='>')

t_17b = ns.nstruct(      # 17B - MSG 31 - Data Block
    (db_hdr,'header'),
    (ns.uint8,'reserved_a'), 
    (ns.uint8,'reserved_b'), 
    (ns.uint16,'n_gates'), # number of gates
    (SI_2,'gate_range'), # gate range to center of first gate - 0.000 to 32.768 km
    (SI_2,'gate_samp_int'), # gate sample interval - 0.25 to 4.0 km
    (SI_2,'tover'), # threshold value for similar to resolution, 0.0 to 20.0 dB
    (SI_2,'SNR_thres'), # signal to noise threshold for valid data, -12.0 to 20.0 dB
    (CODE_1,'ctrl_flags'), # control flags for recombined - 0=none, 1=az rads, 2=range gates, 3=rads and gates to legacy res
    (ns.uint8,'dw_size'), # data word size, 8 or 16
    (ns.single,'scale'), # scale value for data momens int->float
    (ns.single,'offset'), # offset value for data moments int->float
    (t_17i,'data moments'), # data moments                            17I       
    name='t_17b',padding=1,endian='>',base=msg_31_table)

msg_31 = ns.nstruct(       # 17A
    (msg_header,'header'),
    (t_17a,'header_31'), # data
    (msg_31_table[0],'data'), # data
    base=msg_base,name='msg_31',padding=1,endian='>')


In [32]:
# MSG 15 TYPES
t_14a = ns.nstruct(      # 14A - Clutter Filter Map Header  
    (ns.uint16,'date_d'), # days since 1/1/1970
    (ns.uint16,'time_m'), # minutes since midnight
    (ns.uint16,'el_n'), # number of elevations
    name='t_14a',padding=1,endian='>')

t_14_r = ns.nstruct(      # 14A - Clutter Filter Map Range
    (CODE_2,'op_code'), # operation code
    (ns.uint16,'stop_range'), # stop range
    name='t_14_r',padding=1,endian='>')

t_14_az = ns.nstruct(      # 14A - Clutter Filter Map Azimuth
    (ns.uint16,'rz_n'),  # number of range bins in the azimuth
    (t_14_r[0],'ranges'), # ranges
    name='t_14_az',padding=1,endian='>')

t_14_el = ns.nstruct(      # 14A - Clutter Filter Map Elevation
    (t_14_az[0],'azimuths'), # azimuths
    name='t_14_el',padding=1,endian='>')

msg_15 = ns.nstruct(
    (msg_header,'header'),
    (t_14a,'header_15'),
    name='msg_15',padding=1,endian='>')

In [33]:
# message type 2 items

t4_rda_stat_enum = ns.enum('rda_stat_enum',None, CODE_2, bitwise=True,
                        Startup = 0x2, # Startup
                        Standby = 0x4, # Standby
                        Restart = 0x8, # Restart
                        Operate = 0x10, # Operate
                        SpareA = 0x20, # Spare
                        SpareB = 0x40 # Shutdown
)

t4_op_stat_enum = ns.enum('op_stat_enum',None, CODE_2, bitwise=True,
                            Online = 0x2, # Online
                            MaintActReq = 0x4, # Maintenance action required
                            MaintActMand = 0x8, # Maintenance action mandatory
                            CmdShtdwn = 0x10, # Commanded shutdown
                            Inop = 0x20 # Inoperable
)

t4_ctrl_stat_enum = ns.enum('ctrl_stat_enum',None, CODE_2, bitwise=True,
                            LocalOnly = 0x2, # Local control
                            RemoteOnly = 0x4, # Remote control
                            LocRem = 0x8, # Local or remote control
)
t4_aux_pwr_gen_stat_enum = ns.enum('aux_pwr_gen_stat_enum',None, CODE_2, bitwise=True,
                            OnAux = 0x1, # On auxiliary power
                            UtilAvail = 0x2, # Utility power available
                            GenOn = 0x4, # Generator on
                            TfSwitch = 0x8, # Transfer switch - manual
                            CmdSwo = 0x10, # Commanded switch over
)

t4_dat_tx_en_enum = ns.enum('dat_tx_en_enum',None, CODE_2, bitwise=True,
                            Non = 0x2, # None
                            Ref = 0x4, # Reflectivity
                            Vel = 0x8, # Velocity
                            Sw = 0x10, # Spectrum width
)
t4_rda_ctrl_auth_enum = ns.enum('rda_ctrl_auth_enum',None, CODE_2, bitwise=True,
                            NoAct = 0x0, # No action
                            LocalReq = 0x2, # Local control request
                            RemoteReq = 0x4, # Remote control request - local released
)
t4_op_mode_enum = ns.enum('op_mode_enum',None, CODE_2, bitwise=True, Oper = 0x4, Maint = 0x8,)
t4_super_res_stat_enum = ns.enum('super_res_stat_enum',None, CODE_2, bitwise=True,Enabled = 0x2, Disabled = 0x4)
t4_clutter_mit_d_stat_enum = ns.enum('clutter_mit_d_stat_enum',None, CODE_2, bitwise=True,
                                        Disabled = 0x0,
                                        Enabled = 0x1,
                                        SegA = 0x2,
                                        SegB = 0x4,
                                        SegC = 0x8,
                                        SegD = 0x10,
                                        SegE = 0x20)
t4_rda_scan_dat_flag_enum = ns.enum('rda_scan_dat_flag_enum',None, CODE_2, bitwise=True,
                                    AVSET_EN = 0x1,
                                    AVSET_DIS = 0x2,
                                    EBC_EN = 0x4,
                                    RDA_LOG_EN = 0x8,
                                    TIME_SERIES_DAT_REC = 0x10)
t4_rda_alarms_enum = ns.enum('rda_alarms_enum',None, CODE_2, bitwise=True,
                            NoAlarm = 0,
                            TowerUtil = 0x1,
                            Pedestal = 0x2,
                            Tx = 0x4,
                            Rx = 0x8,
                            RDACtrl = 0x10,
                            Comm = 0x20,
                            SigProc = 0x40
)

t4_cmd_ack_enum = ns.enum('cmd_ack_enum',None, CODE_2, bitwise=False,NoAck = 0,RemVCPRec=1,ClutterBypRec=2,ClutterCensorRec=3,RedundChanCmdAcc=4)

t4_chan_ctrl_enum = ns.enum('chan_ctrl_enum',None, CODE_2, bitwise=False,Controlling=0,NonControlling=1)
t4_spot_blank_enum = ns.enum('spot_blank_enum',None, CODE_2, bitwise=True,NotInstalled=0x0,Enabled=0x2,Disabled=0x4)
t4_tps_stat_enum = ns.enum('tps_stat_enum',None, CODE_2, bitwise=False,NotInstalled=0,Off=1,OK=3,Unknown=4)
t4_rms_ctrl_stat_enum = ns.enum('rms_ctrl_stat_enum',None, CODE_2, bitwise=False,NonRMS=0,RMSCtrl=2,RDACtrl=4)
t4_performance_stat_enum = ns.enum('performance_stat_enum',None, CODE_2, bitwise=False,NoCmd=0,ForcePerfPend=1,IP=2)
t4_sig_proc_opt_enum = ns.enum('sig_proc_opt_enum',None, CODE_2, bitwise=False,Enabled=1)
msg_2 = ns.nstruct(      # 4 -- messgae type 2
    (msg_header,'header'),
    (t4_rda_stat_enum,'rda_stat'), # RDA status
    (t4_op_stat_enum,'op_state'), # operability status
    (t4_ctrl_stat_enum,'ctrl_state'), # control status
    (t4_aux_pwr_gen_stat_enum,'aux_pwr_gen_state'), # auxiliary power generator status
    (ns.uint16,'avg_tx_pwr'), # average transmit power
    (SI_2,'ref_calib'), # reflectivity calibration correction
    (t4_dat_tx_en_enum,'dat_tx_en'), # data transmission enabled
    (ns.int16,'vcp'), # volume coverage pattern - 0 = none, pattern sel:  neg = rda local, pos = rda remote
    (t4_rda_ctrl_auth_enum,'rda_ctrl_auth'), # RDA control authority
    (SI_2,'rda_build'), # RDA build number
    (t4_op_mode_enum,'op_mode'), # operation mode
    (t4_super_res_stat_enum,'super_res_stat'), # super resolution status
    (t4_clutter_mit_d_stat_enum,'clutter_mit_d_stat'), # clutter mitigation decision status
    (t4_rda_scan_dat_flag_enum,'rda_scan_dat_flag'), # RDA scan data flag TODO: FIX
    (t4_rda_alarms_enum,'rda_alarms'), # RDA alarms
    (t4_cmd_ack_enum,'cmd_ack'), # command acknowledgement
    (t4_chan_ctrl_enum,'chan_ctrl'), # channel control
    (t4_spot_blank_enum,'spot_blank'), # spot blanking
    (ns.uint16,'bypass_map_date_d'), # bypass map generation date - days since 1/1/1970
    (ns.uint16,'bypass_map_time_m'), # bypass map generation time - minutes since midnight
    (ns.uint16,'clutter_map_date_d'), # clutter map generation date - days since 1/1/1970
    (ns.uint16,'clutter_map_time_m'), # clutter map generation time - minutes since midnight
    (SI_2,'vert_ref_cal_corr'), # vertical calibration correction
    (t4_tps_stat_enum,'tps_stat'), # transition power source status
    (t4_rms_ctrl_stat_enum,'rms_ctrl_stat'), # rms control status
    (t4_performance_stat_enum,'performance_stat'), # performance data collection status
    (ns.uint16[14],'alarm_codes'), # table 4A - 800 possible, 17 at a time
    (t4_sig_proc_opt_enum,'sig_proc_options'), # signal processor options
    (ns.uint16[18],'spares'), # spares
    (ns.int16,'stat_version'), # status version number
    (ns.raw,'padding'), # data
    name='msg_2',padding=1,endian='>')

In [ ]:
# MSG 18 TYPES - TODO
# metadata

In [ ]:
# Message 3 types - TODO

# tons of metadata

In [ ]:
# Message 5 and 7 types - TODO

# sector
# code2 - edge angle
# uint16 - doppler prf num
# uint16 - doppler prf pulse count per rad


# elevation data
# code2 - elev angle 
# code1 - channel config
# code1 - wave form type
# code1 - super res ctrl
# uint8 - surveillance prf num
# uint16 - surveillance prf pulse count per rad
# code2 - az rate 
# ssi2 - reflect snr thresh
# ssi2 - vel snr thresh
# ssi2 - sw snr thresh
# ssi2 - zdr snr thresh
# ssi2 - phidp snr thresh
# ssi2 - rho snr thresh
# ssi2 - cc snr thresh 
# sector - sector 1
# code2 - suplemental data
# sector - sector 2
# code2 - ebc angle
# sector - sector 3
# uint8 - reserved




# msg:
# header
# uint16 - msg size - halfwords
# code2 - pattern type - const elev cut
# uint16 - pattern number -append c, oper or const elev
# uint16 - number of elev cuts
# uint8 - vcp version number
# uint8 - clutter map group number
# code1 - doppler velocity resolution - 0.5/1.0
# code1 - pulse width - long/short
# uint8[2] - reserved
# code2 - vcp sequenceing - elevs, max sails cuts, seq active, truncated vcp
# code2 - vcp supplemental data - saisl vcp, num sails cuts, mrle vcp, num mrle cuts, sparec, mpda vcp, base tilt vcp, num base tilts
# uint8 - reserved
# elev_data[0] - elevation data

In [34]:
# message creator
def create_msg(data):
    r"""
    Create a message of the correct type from the data
    :param data: raw data
    """
    # tpyes to care about right now: {15: 5, 0: 122, 18: 4, 3: 1, 5: 1, 2: 3, 31: 10898}
    type = int.from_bytes(data[15:16], byteorder='big')
    message = None
    if(type==0):
        message = msg_base.create(data)
    elif(type==31):
        message = msg_31.create(data) # TODO: need to parse the data at the pointers
        if(message is None):
            print('bad data')
    elif(type==15):
        message = msg_15.create(data)
    elif(type==2):
        message = msg_2.create(data)
    else:
        message = msg_base.create(data)
    #TODO: parse the message into the correct message type instead of raw data

    return message

In [35]:
# log.setLevel(logging.FATAL-1)
record_count = len(fi.records)
print(f"Number of records: {record_count}")

record = 0
records = RECORD_HOLDER.create('')
for cr in fi.records:
    if(cr.ctrl_word < 0):
        break
    data = bz2.decompress(cr.data[:cr.ctrl_word])
    if data == b'':
        log.error(f"Error decompressing data. CW: {cr.ctrl_word}")
        break


    position = 0 # position in the data
    i = 0 # message counter
    ldm = LDM_UR.create('')
    while(position<len(data)):
        msg_size = int.from_bytes(data[position+12:position+14]) # get size in halfwords
        if(msg_size > 65535):
            log.error(f"Error: Message size too large: {msg_size}")
            raise Exception(f"Stop {msg_size}")
            break
        elif(msg_size<9):
            if (msg_size != 0):
                log.warning(f"Error?: Message size too small: {msg_size}")
                raise Exception(f"Stop {record}.{i}")
            else:
                log.info(f"Message {i} is empty")
                # break
        else:
            pass

        if(record == 0):
            eom = 2432*(i+1)-12 # not really needed but..
        else:
            eom = position+msg_size*2
        
        dat = data[position:eom]
        if dat[0:12] != b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00':
            log.error(f"Error: Prepad {i} is not 0: {dat[0:12]}")
        elif (dat[12:14] == b'\x00\x00' or dat[12:14] == b''):
            log.info(f"in {i}: Product is 0s: {dat[0:28]}")
        else:
            pass
        try:
            
            a_record_that_exists = create_msg(dat)
            ldm.msg.append(a_record_that_exists)
        except Exception as e:
            log.error(f"Error: {e} -  {dat[0:28]}")
            break
        position=eom+12
        i+=1
    records.ldm.append(ldm)
    record+=1
    # log.fatal(f"going to next record {record}")



Number of records: 94


Error: Cannot parse struct: data is corrupted. -  b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00D\x08\x02\x08iM\x99\x04\x0c\x86\x11\x00\x01\x00\x01'
Error: Cannot parse struct: data is corrupted. -  b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00D\x08\x02\x08\x87M\x99\x04\x0c\x8a\x01\x00\x01\x00\x01'


In [36]:
def checkMsg(msg,i,j):
    warns = 0
    infos = 0
    if (msg is None):
        log.error(f"Error: Message {i}.{j} is None")
        return -1
    if msg.header.message_type > 33:
        log.warning(f"Message {i}.{j} type {msg.header.message_type} is not valid!")
        warns+=1
    if msg.header.message_type == 0:
        log.info(f"Message {i}.{j} has type 0. Likely a type 13 legacy area.")
        infos+=1
    if msg.header.milliseconds >= 86400000:
        log.warning(f"Message {i}.{j} time is invalid: {msg.header.milliseconds}")
        warns+=1
    if msg.header.segments > 1 or msg.header.segment > 1:
        log.warning(f"Message {i}.{j} has segments: {msg.header.segment} of {msg.header.segments}")
        warns+=1
    if msg.header.segments == 0 or msg.header.segment == 0:
        log.info(f"Message {i}.{j} has no segments: {msg.header.segment} of {msg.header.segments}")
        infos+=1
    if msg.header.date > 65535:
        log.warning(f"Message {i}.{j} date is invalid: {msg.header.date}")
        warns+=1
    if msg.header.sequence > 65535:
        log.warning(f"Message {i}.{j} sequence is invalid: {msg.header.sequence}")
        warns+=1
    if msg.header.sequence == 0:
        log.info(f"Message {i}.{j} has sequence number 0")
        infos+=1
    if msg.header.channel > 10:
        log.warning(f"Message {i}.{j} redundant channel is invalid: {msg.header.channel}")
        warns+=1
    if msg.header.message_size > 65535:
        log.warning(f"Message {i}.{j} size is invalid: {msg.header.message_size}")
        warns+=1
    if msg.header.prepad[0] != 0 or msg.header.prepad[1] != 0 or msg.header.prepad[2] != 0:
        log.warning(f"Message {i}.{j} Prepad is not 0: {msg.header.prepad}")
        warns+=1
    if(warns > 0):
        log.warning(f"...Message {i}.{j} - W:{warns} - {msg.header.message_type} has sequence number {msg.header.sequence} and is {msg.header.message_size} halfwords")
    type = msg.header.message_type
    if(type is None):
        log.error(f"Error: Message {i}.{j} type is None")
    return type
    

In [37]:

log.setLevel(1000)
prods = {}
totals = 0
for i, ldm in enumerate(records.ldm):
    for j, msg in enumerate(ldm.msg):
        try:
            msgtype = checkMsg(msg,i,j)
            if(msgtype is 2):
                dump = ns.dump(msg,humanread=True,typeinfo='key')
                for k,v in dump['<msg_2>'].items():
                    print(f"{k}: {v}")
            prods[msgtype] = prods.get(msgtype,0)+1
        except Exception as e:
            print(e)
        
        totals+=1
print(prods)
print(totals)
# 3.2.4.17

header: {'<msg_header>': OrderedDict({'prepad': [0, 0, 0], 'message_size': 68, 'channel': 'SINGLE ORDA', 'message_type': 2, 'sequence': 62179, 'date': 19865, 'milliseconds': 67777966, 'segments': 1, 'segment': 1})}
rda_stat: Operate
op_state: Online
ctrl_state: RemoteOnly
aux_pwr_gen_state: UtilAvail
avg_tx_pwr: 1460
ref_calib: OrderedDict({'si': 16})
dat_tx_en: Ref Vel Sw
vcp: 212
rda_ctrl_auth: NoAct
rda_build: OrderedDict({'si': 2210})
op_mode: Oper
super_res_stat: Enabled
clutter_mit_d_stat: Disabled Enabled SegA SegB SegC SegD SegE
rda_scan_dat_flag: AVSET_DIS RDA_LOG_EN TIME_SERIES_DAT_REC 0x20
rda_alarms: NoAlarm
cmd_ack: NoAck
chan_ctrl: Controlling
spot_blank: NotInstalled
bypass_map_date_d: 19865
bypass_map_time_m: 1124
clutter_map_date_d: 19831
clutter_map_time_m: 967
vert_ref_cal_corr: OrderedDict({'si': 25})
tps_stat: OK
rms_ctrl_stat: NonRMS
performance_stat: NoCmd
alarm_codes: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
sig_proc_options: Enabled
spares: [0, 0, 0, 0, 0, 0,

<>:8: SyntaxWarning: "is" with 'int' literal. Did you mean "=="?
<>:8: SyntaxWarning: "is" with 'int' literal. Did you mean "=="?
C:\Users\Max Myzer\AppData\Local\Temp\ipykernel_784676\3059369500.py:8: SyntaxWarning: "is" with 'int' literal. Did you mean "=="?
  if(msgtype is 2):
